In [1]:
import sys
sys.path += ["/home/yousuf/workspace/dissertation/code/rPPG/python/core"]

In [2]:
from region_selection import BayesianSkinDetector
import helper

In [ ]:
images = helper.get_cropped_test_images()
helper.show_images_plt(images)

In [ ]:
heatmaps = [BayesianSkinDetector(skin_std=40, non_skin_std=150).detect(i) for i in images]

In [ ]:
import numpy as np
for t in heatmaps:
    x,v= t
#     print(x)
#     print(y)
    print("Skin posterior")
    helper.display_heatmap([x])
#     print("Not skin posterior")
#     helper.display_heatmap([y])
#     print(z)
    print("Mask above mean")
    helper.display_heatmap([x>np.mean(x)])
    print("Top 70% of values")
    helper.display_heatmap([x>np.percentile(x, 30)])
    print("Top 80% of values")
    helper.display_heatmap([x>np.percentile(x, 20)])
    print("Mean value")
    print(v)


In [ ]:
import matplotlib
import matplotlib.cm as cm
cmap = cm.get_cmap('RdBu')

In [ ]:
list(map(cmap, [0,0.5, 0.6]))

In [ ]:
import numpy as np
distribution = np.array([[0.1, 0.4],[0.5, 0]])
image = np.array([[[10,10,10],[20,20,20]],[[100,100,100], [10000,10000,10000]]])
print(np.sum(image[:,:,0]*distribution))
# np.broadcast(distribution,image)

In [ ]:
import region_selection
region_selection.weighted_mean(image, distribution)

In [ ]:
distribution = np.array([[1, 0],[1, 0]])
region_selection.weighted_mean(image, distribution)

In [ ]:
import pipeline
from configuration import Configuration
from face_det import KLTBoxingWithThresholding, DNNDetector
from region_selection import BayesianSkinDetector
from hr_isolator import ICAProcessor

config = Configuration(KLTBoxingWithThresholding(DNNDetector()),  BayesianSkinDetector(), ICAProcessor(), 1200, 60)
pipeline.tracking_pipeline("test-face-detection-videos/mov-1.mp4",config,display=True)

In [70]:
from visualisation import Visualiser
from face_det import DNNDetector
from face_det import KLTBoxingWithThresholding, DNNDetector
from region_selection import BayesianSkinDetector
from hr_isolator import ICAProcessor
from region_selection import BayesianSkinDetector
import helper
im = helper.get_test_images()[0]
cr = helper.get_cropped_test_images()[0]
hmap, value = BayesianSkinDetector(skin_std=40, non_skin_std=150).detect(cr) 
x = display(im, DNNDetector().detect_face(im), hmap)
x = [np.array(x[i], dtype=np.uint8) for i in range(4)]
helper.show_images_plt(x)

[[0.18588674 0.19640163 0.19640163 ... 0.1523237  0.16160308 0.16160308]
 [0.18588674 0.20024139 0.19640163 ... 0.16160308 0.1523237  0.1523237 ]
 [0.18588674 0.18588674 0.18588674 ... 0.16160308 0.16160308 0.16160308]
 ...
 [0.1725142  0.18257402 0.18257402 ... 0.17558967 0.17558967 0.18583004]
 [0.1725142  0.1725142  0.1725142  ... 0.17558967 0.17558967 0.17558967]
 [0.1725142  0.1725142  0.18257402 ... 0.18583004 0.18583004 0.18583004]]


In [69]:
import numpy as np
import cv2 as cv
import matplotlib
import matplotlib.cm as cm
def display(frame, faces, area_of_interest):
    height, width, _ = frame.shape
    x,y,w,h = faces[0]
    cmap = cm.get_cmap('winter')
    print(area_of_interest)

    area_of_interest = np.pad(area_of_interest, ((y,height-(y+h)),(x,width-(x+w))), 'constant', constant_values=0)
    mask_of_roi = np.pad(np.ones(shape=(h,w)), ((y,height-(y+h)),(x,width-(x+w))), 'constant', constant_values=0)
    h,w = area_of_interest.shape
    rectangle = 255*np.array(list(map(cmap, area_of_interest)))
    rectangle = rectangle.reshape((h,w,4))
    mask_of_roi = np.repeat(mask_of_roi[:, :, np.newaxis], 3, axis=2)

    # Overlay the points being considered and the rectangle of the face
    # rectangle = np.full(shape=(height,width,3), fill_value=[0,255,0], dtype=np.uint8)
    rectangle = rectangle[:,:,:3]
    # Need to take and with one vector since otherwise we get 254 instead of the desired value of 0 at each point outside the mask
    # i.e. NOT(0000 0001) = 1111 1110 rather than 0
    # mask_of_roi = cv.bitwise_and(cv.bitwise_not(area_of_interest), np.ones(shape=(height,width,3), dtype=np.uint8))
    alpha = 0.8
    foreground = rectangle.astype(float)
    background = frame.astype(float)
    mask_of_roi = mask_of_roi.astype(float)/2
    # print(mask_of_roi.shape)
    # print(rectangle.shape)
    foreground = cv.multiply(mask_of_roi, foreground)
    background = cv.multiply(1.0-mask_of_roi, background)
    blended = cv.addWeighted(foreground, alpha, background, 1, 0)
    return rectangle, foreground, background, blended
    # self.out.write(np.uint8(blended))